In [1]:
import requests as req
import pandas as pd
import sqlite3

In [2]:
ENDPOINT = 'https://api.hh.ru/vacancies'
DB_NAME = 'api.db'
TABLE_NAME = 'vacancies'

In [3]:

api_params = {'text':'python middle',
              'per_page':25,
              'page':0}
vacancyDetailInfo =[]

In [4]:
def parsJsonAppendData (data):
    for result in data['items']:
        vacancyRow = []
        listOfRequirements = []
        vacancyName = result['name']
        companyName = result['employer']['name']
        vacancyid = result['id']
        vacancyDetail = req.get(ENDPOINT+'/'+vacancyid).json()
        vacancyDescription = vacancyDetail['description']
        for skill in vacancyDetail['key_skills']:
            listOfRequirements.append(skill['name'])
        vacancyRow.append(companyName)
        vacancyRow.append(vacancyName)
        vacancyRow.append(vacancyDescription)
        vacancyRow.append(listOfRequirements)
        vacancyDetailInfo.append(vacancyRow)

In [5]:
while (True):
    results = req.get(ENDPOINT, params = api_params).json()
    if (len(results['items']) == 0):
        break
    parsJsonAppendData(results)
    print('Записей в итоговом результате',len(vacancyDetailInfo))
    api_params['page'] += 1
    

Записей в итоговом результате 25
Записей в итоговом результате 50
Записей в итоговом результате 75
Записей в итоговом результате 100
Записей в итоговом результате 125
Записей в итоговом результате 150
Записей в итоговом результате 175
Записей в итоговом результате 200
Записей в итоговом результате 225
Записей в итоговом результате 250
Записей в итоговом результате 275
Записей в итоговом результате 300
Записей в итоговом результате 325
Записей в итоговом результате 350
Записей в итоговом результате 375
Записей в итоговом результате 400
Записей в итоговом результате 425
Записей в итоговом результате 450
Записей в итоговом результате 475
Записей в итоговом результате 500
Записей в итоговом результате 525
Записей в итоговом результате 550
Записей в итоговом результате 575
Записей в итоговом результате 600
Записей в итоговом результате 625
Записей в итоговом результате 650
Записей в итоговом результате 675
Записей в итоговом результате 700
Записей в итоговом результате 725
Записей в итогово

In [6]:
table_columns = ['company_name', 'position', 'job_description', 'key_skills']
df = pd.DataFrame(data=vacancyDetailInfo,columns=table_columns,index=None)

In [7]:
connection = sqlite3.connect(DB_NAME)
df.to_sql(TABLE_NAME,connection,if_exists='append',index=False)

InterfaceError: Error binding parameter 3 - probably unsupported type.

In [27]:
connection = sqlite3.connect(DB_NAME)
cursor = connection.cursor()
for line in vacancyDetailInfo:
    insert_several_rows_parameters = f'INSERT INTO {TABLE_NAME} (company_name, position, job_description, key_skills) VALUES ("{line[0]}", "{line[1]}","{line[2]}", "{line[3]}")'
    cursor.execute(insert_several_rows_parameters)


OperationalError: database is locked